In [1]:
import pandas as pd
import xgboost as xgb
import time
import numpy as np
from sklearn import preprocessing
import category_encoders as ce

In [2]:
entbase_df = pd.read_csv('./data/1entbase.csv')
alter_df = pd.read_csv('./data/2alter.csv')
branch_df = pd.read_csv('./data/3branch.csv')
invest_df = pd.read_csv('./data/4invest.csv')
right_df = pd.read_csv('./data/5right.csv')
project_df = pd.read_csv('./data/6project.csv')
lawsuit_df = pd.read_csv('./data/7lawsuit.csv')
breakfaith_df = pd.read_csv('./data/8breakfaith.csv')
recruit_df = pd.read_csv('./data/9recruit.csv')
qualification_df = pd.read_csv('./data/10qualification.csv')

c:\users\linjinjin\anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def translate_year(date):
    year = int(date[:4])
    month = int(date[-2:])
    return (year-2010)*12 + month

def get_interaction_feature(df, feature_A, feature_B):
    feature_A_list = sorted(df[feature_A].unique())
    feature_B_list = sorted(df[feature_B].unique())
    count = 0
    mydict = {}
    for i in feature_A_list:
        mydict[int(i)] = {}
        for j in feature_B_list:
            mydict[int(i)][int(j)] = count
            count += 1
    return df.apply(lambda x: mydict[int(x[feature_A])][int(x[feature_B])], axis=1)

In [4]:
import math

def get_entbase_feature():
    entbase = entbase_df.copy()
    entbase.MPNUM.fillna(0, inplace=True)
    entbase.INUM.fillna(0, inplace=True)
    entbase.FINZB.fillna(0, inplace=True)
    entbase.FSTINUM.fillna(0, inplace=True)
    entbase.TZINUM.fillna(0, inplace=True)
    entbase.ENUM.fillna(0, inplace=True)
    entbase.HY.fillna(51, inplace=True)
    entbase.ZCZB.fillna(7, inplace=True)
    
#     entbase["PROV"] = preprocessing.LabelEncoder().fit_transform(entbase.PROV.values)
    entbase["RGYEAR"] = 2015 - entbase["RGYEAR"]
    entbase["INDEX_SUM"] = entbase["MPNUM"] + entbase["INUM"] + entbase["MPNUM"] + entbase["TZINUM"] + entbase["ENUM"]
    entbase["ZCZB_FINZB_RATE"] = entbase["ZCZB"] / entbase["FINZB"]
    entbase["ZCZB_FINZB_RATE"] = entbase["FINZB"] / entbase["ZCZB"]
    
    entbase["DIVIDE_ZCZB_RGYEAR"] = entbase["ZCZB"] / entbase["RGYEAR"]
    entbase["DIVIDE_RGYEAR_ZCZB"] = entbase["RGYEAR"] / entbase["ZCZB"]
    entbase["MUL_HY_PROV"] = entbase["HY"] * entbase["PROV"]
    entbase["MUL_HY_ETYPE"] = entbase["HY"] * entbase["ETYPE"]
    entbase["MUL_HY_RGYEAR"] = entbase["HY"] * entbase["RGYEAR"]
    
    entbase["RGYEAR_ZCZB"] = get_interaction_feature(entbase, "RGYEAR", "ZCZB")
    entbase["RGYEAR_FINZB"] = get_interaction_feature(entbase, "RGYEAR", "FINZB")
    entbase["RGYEAR_INUM"] = get_interaction_feature(entbase, "RGYEAR", "INUM")
    entbase["RGYEAR_ENUM"] = get_interaction_feature(entbase, "RGYEAR", "ENUM")
    entbase["RGYEAR_FSTINUM"] = get_interaction_feature(entbase, "RGYEAR", "FSTINUM")
    entbase["RGYEAR_TZINUM"] = get_interaction_feature(entbase, "RGYEAR", "TZINUM")
    entbase["RGYEAR_MPNUM"] = get_interaction_feature(entbase, "RGYEAR", "MPNUM")
    
    #new 12.7
    entbase["MUL_PROV_HY"] = entbase["PROV"] * entbase["HY"]
    entbase["MUL_PROV_ETYPE"] = entbase["PROV"] * entbase["ETYPE"]
    entbase["MUL_PROV_INUM"] = entbase["PROV"] * entbase["INUM"]
    entbase["MUL_PROV_INUM"] = entbase["PROV"] * entbase["ENUM"]
    entbase["MUL_PROV_INUM"] = entbase["PROV"] * entbase["MPNUM"]
    entbase["MUL_PROV_INUM"] = entbase["PROV"] * entbase["FSTINUM"]
    entbase["MUL_PROV_INUM"] = entbase["PROV"] * entbase["TZINUM"]
#     entbase["MUL_PROV_INUM"] = entbase["PROV"] * entbase["RGYEAR"]
#     entbase["MUL_PROV_INUM"] = entbase["PROV"] * entbase["ZCZB"]
    
    #new 12.1
    entbase["ZB_SUM"] = entbase["ZCZB"] + entbase["FINZB"]

    #12.10
    tmp = entbase.ZCZB.value_counts().reset_index()
    tmp.columns = ['ZCZB', 'ZCZB_COUNT']
    entbase = pd.merge(entbase, tmp, on="ZCZB", how="left")
    
    tmp = entbase.RGYEAR.value_counts().reset_index()
    tmp.columns = ['RGYEAR', 'RGYEAR_COUNT']
    entbase = pd.merge(entbase, tmp, on="RGYEAR", how="left")
    
    for column in ["MPNUM", "INUM", "FINZB", "FSTINUM", "TZINUM", "ENUM", "ZCZB", "INDEX_SUM", "RGYEAR"]:
#         groupby_list = [["HY"], ["ETYPE"], ["HY", "ETYPE"], ["HY", "PROV"], ["ETYPE", "PROV"]]
        groupby_list = [["HY"], ["ETYPE"], ["PROV"], ["HY", "PROV"], ["ETYPE", "PROV"], ["ZCZB_COUNT"], ["RGYEAR_COUNT"]]
        for groupby in groupby_list:
            groupby_keylist = []
            for key in groupby:
                groupby_keylist.append(entbase[key])
                    
            tmp = entbase[column].groupby(groupby_keylist).mean().reset_index()
            tmp.columns = np.append(groupby, ["MEAN_COLUMN"])
            entbase = pd.merge(entbase, tmp, on=groupby, how="left")
            entbase["_".join(groupby) + "_" + column + "_MEAN_GAP"] = entbase[column] - entbase["MEAN_COLUMN"]
            entbase["_".join(groupby) + "_" + column + "_RATE"] = entbase[column] / entbase["MEAN_COLUMN"]
            entbase.drop(["MEAN_COLUMN"], axis=1, inplace=True)
            
            tmp = entbase[column].groupby(groupby_keylist).min().reset_index()
            tmp.columns = np.append(groupby, ["MIN_COLUMN"])
            entbase = pd.merge(entbase, tmp, on=groupby, how="left")
            entbase["_".join(groupby) + "_" + column + "_MIN_GAP"] = entbase[column] - entbase["MIN_COLUMN"]
            entbase.drop(["MIN_COLUMN"], axis=1, inplace=True)
            
            tmp = entbase[column].groupby(groupby_keylist).max().reset_index()
            tmp.columns = np.append(groupby, ["MAX_COLUMN"])
            entbase = pd.merge(entbase, tmp, on=groupby, how="left")
            entbase["_".join(groupby) + "_" + column + "_MAX_GAP"] = entbase[column] - entbase["MAX_COLUMN"]
            entbase.drop(["MAX_COLUMN"], axis=1, inplace=True)
            
            tmp = entbase[column].groupby(groupby_keylist).sum().reset_index()
            tmp.columns = np.append(groupby, ["SUM_COLUMN"])
            entbase = pd.merge(entbase, tmp, on=groupby, how="left")
            entbase["_".join(groupby) + "_" + column + "_SUM_GAP"] = entbase[column] / entbase["SUM_COLUMN"]
            entbase.drop(["SUM_COLUMN"], axis=1, inplace=True)
        
    entbase.drop(['RGYEAR_COUNT', 'ZCZB_COUNT'], axis = 1, inplace = True)
    #new 12.7
#     def divided_year(x) :
#         if x < 10:
#             return 1
#         if x < 20:
#             return 2
#         if x < 50:
#             return 3
#         if x < 100:
#             return 4
#         if x < 200:
#             return 5
#         if x < 500:
#             return 6
#         if  x < 1000:
#             return 7
#         if  x < 2000:
#             return 8
#         if  x < 5000:
#             return 9
#         if  x < 100000:
#             return 10
#         else:
#             return 11
    
#     entbase['FSTINUM_MUL_LOGZCZB'] = entbase['FSTINUM'] * entbase['ZCZB'].apply(lambda x : math.log(x + 1))
#     entbase['RGYEAR_MUL_ZCZB'] = entbase['RGYEAR'] * entbase['ZCZB']
#     entbase['RGYEAR_MUL_ZCZB'] = entbase.RGYEAR_MUL_ZCZB.apply(divided_year)
    
#     tmp = entbase['FSTINUM_MUL_LOGZCZB'].groupby(entbase.HY).agg(np.mean).reset_index()
#     tmp.columns = ["HY", "FSTINUM_MUL_LOGZCZB_HY_MEAN"]
#     entbase = pd.merge(entbase, tmp, on="HY", how="left")
#     tmp = entbase['FSTINUM_MUL_LOGZCZB'].groupby(entbase.ETYPE).agg(np.mean).reset_index()
#     tmp.columns = ["ETYPE", "FSTINUM_MUL_LOGZCZB_ETYPE_MEAN"]
#     entbase = pd.merge(entbase, tmp, on="ETYPE", how="left")
#     tmp = entbase['FSTINUM_MUL_LOGZCZB'].groupby(entbase.PROV).agg(np.mean).reset_index()
#     tmp.columns = ["PROV", "FSTINUM_MUL_LOGZCZB_PROV_MEAN"]
#     entbase = pd.merge(entbase, tmp, on="PROV", how="left")
#     entbase['FSTINUM_MUL_LOGZCZB_HY_GAP'] = entbase['FSTINUM_MUL_LOGZCZB'] - entbase["FSTINUM_MUL_LOGZCZB_HY_MEAN"]
#     entbase['FSTINUM_MUL_LOGZCZB_ETYPE_GAP'] = entbase['FSTINUM_MUL_LOGZCZB'] - entbase["FSTINUM_MUL_LOGZCZB_ETYPE_MEAN"]
#     entbase['FSTINUM_MUL_LOGZCZB_PROV_GAP'] = entbase['FSTINUM_MUL_LOGZCZB'] - entbase["FSTINUM_MUL_LOGZCZB_PROV_MEAN"]
    
#     tmp = entbase['MPNUM'].groupby(entbase.RGYEAR_MUL_ZCZB).agg([np.mean, np.std]).reset_index()
#     tmp.columns = ["RGYEAR_MUL_ZCZB", "RGYEAR_MUL_ZCZB_MPNUM_MEAN", 'RGYEAR_MUL_ZCZB_MPNUM_STD']
#     entbase = pd.merge(entbase, tmp, on="RGYEAR_MUL_ZCZB", how="left")
    
#     tmp = entbase['INUM'].groupby(entbase.RGYEAR_MUL_ZCZB).agg([np.mean, np.std]).reset_index()
#     tmp.columns = ["RGYEAR_MUL_ZCZB", "RGYEAR_MUL_ZCZB_INUM_MEAN", 'RGYEAR_MUL_ZCZB_INUM_STD']
#     entbase = pd.merge(entbase, tmp, on="RGYEAR_MUL_ZCZB", how="left")
    
#     tmp = entbase['FINZB'].groupby(entbase.RGYEAR_MUL_ZCZB).agg([np.mean, np.std]).reset_index()
#     tmp.columns = ["RGYEAR_MUL_ZCZB", "RGYEAR_MUL_ZCZB_FINZB_MEAN", "RGYEAR_MUL_ZCZB_FINZB_STD"]
#     entbase = pd.merge(entbase, tmp, on="RGYEAR_MUL_ZCZB", how="left")
    
#     tmp = entbase['FSTINUM'].groupby(entbase.RGYEAR_MUL_ZCZB).agg([np.mean, np.std]).reset_index()
#     tmp.columns = ["RGYEAR_MUL_ZCZB", "RGYEAR_MUL_ZCZB_FSTINUM_MEAN", "RGYEAR_MUL_ZCZB_FSTINUM_STD"]
#     entbase = pd.merge(entbase, tmp, on="RGYEAR_MUL_ZCZB", how="left")
    
#     tmp = entbase['TZINUM'].groupby(entbase.RGYEAR_MUL_ZCZB).agg([np.mean, np.std]).reset_index()
#     tmp.columns = ["RGYEAR_MUL_ZCZB", "RGYEAR_MUL_ZCZB_TZINUM_MEAN", "RGYEAR_MUL_ZCZB_TZINUM_STD"]
#     entbase = pd.merge(entbase, tmp, on="RGYEAR_MUL_ZCZB", how="left")
    
#     tmp = entbase['ENUM'].groupby(entbase.RGYEAR_MUL_ZCZB).agg([np.mean, np.std]).reset_index()
#     tmp.columns = ["RGYEAR_MUL_ZCZB", "RGYEAR_MUL_ZCZB_ENUM_MEAN","RGYEAR_MUL_ZCZB_ENUM_STD"]
#     entbase = pd.merge(entbase, tmp, on="RGYEAR_MUL_ZCZB", how="left")
    
#     entbase["RGYEAR_MUL_ZCZB_MPNUM_GAP"] = entbase["MPNUM"] - entbase["RGYEAR_MUL_ZCZB_MPNUM_MEAN"]
#     entbase["RGYEAR_MUL_ZCZB_INUM_GAP"] = entbase['INUM'] - entbase["RGYEAR_MUL_ZCZB_INUM_MEAN"]
#     entbase["RGYEAR_MUL_ZCZB_FINZB_GAP"] = entbase['FINZB'] - entbase["RGYEAR_MUL_ZCZB_FINZB_MEAN"]
#     entbase["RGYEAR_MUL_ZCZB_FSTINUM_GAP"] = entbase["FSTINUM"] - entbase["RGYEAR_MUL_ZCZB_FSTINUM_MEAN"]
#     entbase["RGYEAR_MUL_ZCZB_TZINUM_GAP"] = entbase["TZINUM"] - entbase["RGYEAR_MUL_ZCZB_TZINUM_MEAN"]
#     entbase["RGYEAR_MUL_ZCZB_ENUM_GAP"] = entbase["ENUM"] - entbase["RGYEAR_MUL_ZCZB_ENUM_MEAN"]
        
    #new 12.6
    #entbase = entbase.join(pd.get_dummies(entbase["HY"], prefix='HY'))
#     entbase = entbase.join(pd.get_dummies(entbase["ETYPE"], prefix='ETYPE'))
#    entbase = entbase.join(pd.get_dummies(entbase["PROV"], prefix='PROV'))
#     entbase.drop(["HY", "ETYPE", "PROV"], axis=1, inplace=True)
#    entbase = entbase.join(pd.get_dummies(entbase["HY"], prefix='HY_RGYEAR').multiply(entbase["RGYEAR"], axis="index"))
#    entbase = entbase.join(pd.get_dummies(entbase["HY"], prefix='HY_ZCZB').multiply(entbase["ZCZB"], axis="index"))
#    entbase = entbase.join(pd.get_dummies(entbase["ETYPE"], prefix='ETYPE_RGYEAR').multiply(entbase["RGYEAR"], axis="index"))
#    entbase = entbase.join(pd.get_dummies(entbase["ETYPE"], prefix='ETYPE_ZCZB').multiply(entbase["ZCZB"], axis="index"))
#     entbase = entbase.join(pd.get_dummies(entbase["PROV"], prefix='PROV_ZCZB').multiply(entbase["ZCZB"], axis="index"))
#     entbase.drop(["HY", "ETYPE", "PROV"], axis=1, inplace=True)
#     entbase = entbase.join(pd.get_dummies(entbase["HY"], prefix='HY_RGYEAR_ZCZB').multiply(entbase["RGYEAR_ZCZB"], axis="index"))
#     entbase.drop(["PROV", "HY", "ETYPE"], axis=1, inplace=True)
    return entbase

entbase_feat = get_entbase_feature()
entbase_feat.head()

,PROV,EID,RGYEAR,HY,ZCZB,ETYPE,MPNUM,INUM,ENUM,FINZB,...,ZCZB_COUNT_RGYEAR_MEAN_GAP,ZCZB_COUNT_RGYEAR_RATE,ZCZB_COUNT_RGYEAR_MIN_GAP,ZCZB_COUNT_RGYEAR_MAX_GAP,ZCZB_COUNT_RGYEAR_SUM_GAP,RGYEAR_COUNT_RGYEAR_MEAN_GAP,RGYEAR_COUNT_RGYEAR_RATE,RGYEAR_COUNT_RGYEAR_MIN_GAP,RGYEAR_COUNT_RGYEAR_MAX_GAP,RGYEAR_COUNT_RGYEAR_SUM_GAP
0,11,p13836,35,51.0,480.0,13,25.0,1.0,1.0,98041.0,...,27.287582,4.538136,35,-1,0.029661,0.0,1.0,0,0,0.007519
1,11,p49137,22,74.0,30.0,13,0.0,1.0,1.0,570.0,...,15.049413,3.165200,22,-37,0.000232,0.0,1.0,0,0,0.000464
2,11,p108756,0,69.0,1000.0,6,2.0,1.0,1.0,1000.0,...,-4.446696,0.000000,0,-65,0.000000,0.0,NaN,0,0,NaN
3,11,p7320,10,74.0,240.0,5,0.0,2.0,2.0,1500.0,...,-1.619048,0.860656,10,-46,0.004554,0.0,1.0,0,0,0.000076
4,11,p194070,1,51.0,100000.0,3,3.0,2.0,2.0,1000.0,...,-3.491228,0.222656,1,-61,0.001953,0.0,1.0,0,0,0.000013


In [5]:

def translate_money(x):
    if isinstance(x, float): return x
    if x == -1:
        return np.nan
    elif x[-18:] == ' (单位：万元)':
        return float(x[:-18])
    elif x[-9:] == '万美元':
        return float(x[:-9])
    elif x[-3:] == '万':
        return float(x[:-3])
    elif x[-12:] == '万人民币':
        return float(x[:-12])
    elif (x[-6:] == '万元'):
        if x[:-6] == 'null':
            return np.nan
        elif x[:9] == '人民币':
            return float(x[9:-6])
        elif (x[:6] == '美元') | (x[:6] == '港币'):
            return float(x[6:-6]) 
        else:
            return float(x[:-6])
    elif x[-9:] == '万港元':
        return float(x[:-9])
    else:
        return float(x)

def get_alter_feature():
    alter = alter_df.copy()
    
    alter["ALTDATE"] = alter["ALTDATE"].apply(translate_year)
    alter["ALTAF"] = alter["ALTAF"].apply(translate_money)
    alter["ALTBE"] = alter["ALTBE"].apply(translate_money)
    alter["ALT_BE_AF_GAP"] = alter["ALTAF"] - alter["ALTBE"]
    
    alter_feat = alter["ALTERNO"].groupby(alter.EID).count().reset_index()
    alter_feat.columns = ["EID", "ALTER_COUNT"]
    
    tmp = alter["ALTERNO"].groupby(alter.EID).nunique().reset_index()
    tmp.columns = ["EID", "ALTER_UNIQUE_COUNT"]
    alter_feat = pd.merge(alter_feat, tmp, on="EID", how="left")
    
    tmp = alter["ALTERNO"].groupby([alter.EID, alter.ALTERNO]).count().unstack().reset_index()#.fillna(0)
    tmp.columns = [i if i == 'EID' else 'ALTERNO_'+str(i) for i in tmp.columns]
    alter_feat = pd.merge(alter_feat, tmp, on='EID', how='left')
    for column in tmp.columns:
        if column != "EID":
            alter_feat[column + "_RATE"] = alter_feat[column] / alter_feat["ALTER_COUNT"]
    
    tmp = alter["ALTDATE"].groupby(alter.EID).agg([min, max, np.ptp, np.std]).reset_index()
    tmp.columns = [i if i == 'EID' else 'ALTDATE_'+str(i) for i in tmp.columns]
    alter_feat = pd.merge(alter_feat, tmp, on="EID", how="left")
    
    tmp = alter[alter.ALTERNO == "05"]["ALTBE"].groupby(alter.EID).agg([min, max, np.mean, np.std, np.ptp]).reset_index()
    tmp.columns = [i if i == 'EID' else 'ALTBE_05_'+str(i) for i in tmp.columns]
    alter_feat = pd.merge(alter_feat, tmp, on="EID", how="left")
    
    tmp = alter[alter.ALTERNO == "05"]["ALTAF"].groupby(alter.EID).agg([min, max, np.mean, np.std, np.ptp]).reset_index()
    tmp.columns = [i if i == 'EID' else 'ALTAF_05_'+str(i) for i in tmp.columns]
    alter_feat = pd.merge(alter_feat, tmp, on="EID", how="left")
    
    tmp = alter[alter.ALTERNO == "27"]["ALTBE"].groupby(alter.EID).agg([min, max, np.mean, np.std, np.ptp]).reset_index()
    tmp.columns = [i if i == 'EID' else 'ALTBE_27_'+str(i) for i in tmp.columns]
    alter_feat = pd.merge(alter_feat, tmp, on="EID", how="left")
    
    tmp = alter[alter.ALTERNO == "27"]["ALTAF"].groupby(alter.EID).agg([min, max, np.mean, np.std, np.ptp]).reset_index()
    tmp.columns = [i if i == 'EID' else 'ALTAF_27_'+str(i) for i in tmp.columns]
    alter_feat = pd.merge(alter_feat, tmp, on="EID", how="left")
    
    tmp = alter[(alter.ALTERNO == "05") | (alter.ALTERNO == "27")]["ALTBE"].groupby(alter.EID).agg([min, max, np.mean, np.std, np.ptp]).reset_index()
    tmp.columns = [i if i == 'EID' else 'ALTBE_05_27_'+str(i) for i in tmp.columns]
    alter_feat = pd.merge(alter_feat, tmp, on="EID", how="left")
    
    tmp = alter[(alter.ALTERNO == "05") | (alter.ALTERNO == "27")]["ALTAF"].groupby(alter.EID).agg([min, max, np.mean, np.std, np.ptp]).reset_index()
    tmp.columns = [i if i == 'EID' else 'ALTAF_05_27_'+str(i) for i in tmp.columns]
    alter_feat = pd.merge(alter_feat, tmp, on="EID", how="left")
    
    tmp = alter[alter.ALTERNO == "05"]["ALT_BE_AF_GAP"].groupby(alter.EID).agg([min ,max, np.mean]).reset_index()
    tmp.columns = [i if i == 'EID' else 'ALT_05_BE_AF_GAP'+str(i) for i in tmp.columns]
    alter_feat = pd.merge(alter_feat, tmp, on="EID", how="left")
    
    tmp = alter[alter.ALTERNO == "27"]["ALT_BE_AF_GAP"].groupby(alter.EID).agg([min ,max, np.mean]).reset_index()
    tmp.columns = [i if i == 'EID' else 'ALT_27_BE_AF_GAP'+str(i) for i in tmp.columns]
    alter_feat = pd.merge(alter_feat, tmp, on="EID", how="left")
    
    tmp = alter[(alter.ALTERNO == "05") | (alter.ALTERNO == "27")]["ALT_BE_AF_GAP"].groupby(alter.EID).agg([min ,max, np.mean]).reset_index()
    tmp.columns = [i if i == 'EID' else 'ALT_05_27_BE_AF_GAP'+str(i) for i in tmp.columns]
    alter_feat = pd.merge(alter_feat, tmp, on="EID", how="left")
    
    #####################################################################################################
    #11.29
    alter = pd.merge(alter, entbase_df[["EID", "HY"]], on="EID", how="left")
    alter_feat = pd.merge(alter_feat, entbase_df[["EID", "HY"]], on="EID", how="left")
    
    tmp = alter[alter.ALTERNO == "05"]["ALTBE"].groupby(alter.HY).mean().reset_index()
    tmp.columns = ["HY", "HY_05_ALTBE_MEAN"]
    alter_feat = pd.merge(alter_feat, tmp, on="HY", how="left")
    alter_feat["HY_05_ALTBE_MEAN_GAP"] = alter_feat["ALTBE_05_mean"] - alter_feat["HY_05_ALTBE_MEAN"]
    alter_feat = alter_feat.drop(["HY_05_ALTBE_MEAN"], axis=1)
    
    tmp = alter[alter.ALTERNO == "05"]["ALTAF"].groupby(alter.HY).mean().reset_index()
    tmp.columns = ["HY", "HY_05_ALTAF_MEAN"]
    alter_feat = pd.merge(alter_feat, tmp, on="HY", how="left")
    alter_feat["HY_05_ALTAF_MEAN_GAP"] = alter_feat["ALTAF_05_mean"] - alter_feat["HY_05_ALTAF_MEAN"]
    alter_feat = alter_feat.drop(["HY_05_ALTAF_MEAN"], axis=1)
    
    tmp = alter[alter.ALTERNO == "27"]["ALTBE"].groupby(alter.HY).mean().reset_index()
    tmp.columns = ["HY", "HY_27_ALTBE_MEAN"]
    alter_feat = pd.merge(alter_feat, tmp, on="HY", how="left")
    alter_feat["HY_27_ALTBE_MEAN_GAP"] = alter_feat["ALTBE_27_mean"] - alter_feat["HY_27_ALTBE_MEAN"]
    alter_feat = alter_feat.drop(["HY_27_ALTBE_MEAN"], axis=1)
    
    tmp = alter[alter.ALTERNO == "05"]["ALTAF"].groupby(alter.HY).mean().reset_index()
    tmp.columns = ["HY", "HY_27_ALTAF_MEAN"]
    alter_feat = pd.merge(alter_feat, tmp, on="HY", how="left")
    alter_feat["HY_27_ALTAF_MEAN_GAP"] = alter_feat["ALTAF_27_mean"] - alter_feat["HY_27_ALTAF_MEAN"]
    alter_feat = alter_feat.drop(["HY_27_ALTAF_MEAN"], axis=1)
    
    tmp = alter[(alter.ALTERNO == "05") | (alter.ALTERNO == "27")]["ALTBE"].groupby(alter.HY).mean().reset_index()
    tmp.columns = ["HY", "HY_05_27_ALTBE_MEAN"]
    alter_feat = pd.merge(alter_feat, tmp, on="HY", how="left")
    alter_feat["HY_05_27_ALTBE_MEAN_GAP"] = alter_feat["ALTBE_05_27_mean"] - alter_feat["HY_05_27_ALTBE_MEAN"]
    alter_feat = alter_feat.drop(["HY_05_27_ALTBE_MEAN"], axis=1)
    
    tmp = alter[(alter.ALTERNO == "05") | (alter.ALTERNO == "27")]["ALTAF"].groupby(alter.HY).mean().reset_index()
    tmp.columns = ["HY", "HY_05_27_ALTAF_MEAN"]
    alter_feat = pd.merge(alter_feat, tmp, on="HY", how="left")
    alter_feat["HY_05_27_ALTAF_MEAN_GAP"] = alter_feat["ALTAF_05_27_mean"] - alter_feat["HY_05_27_ALTAF_MEAN"]
    alter_feat = alter_feat.drop(["HY_05_27_ALTAF_MEAN"], axis=1)
    
    alter_feat = alter_feat.drop(["HY"], axis=1)
    #####################################################################################################
    
    alter = pd.merge(alter, entbase_df[["EID", "PROV"]], on="EID", how="left")
    alter_feat = pd.merge(alter_feat, entbase_df[["EID", "PROV"]], on="EID", how="left")
    
    tmp = alter[alter.ALTERNO == "05"]["ALTBE"].groupby(alter.PROV).mean().reset_index()
    tmp.columns = ["PROV", "PROV_05_ALTBE_MEAN"]
    alter_feat = pd.merge(alter_feat, tmp, on="PROV", how="left")
    alter_feat["PROV_05_ALTBE_MEAN_GAP"] = alter_feat["ALTBE_05_mean"] - alter_feat["PROV_05_ALTBE_MEAN"]
    alter_feat = alter_feat.drop(["PROV_05_ALTBE_MEAN"], axis=1)
    
    tmp = alter[alter.ALTERNO == "05"]["ALTAF"].groupby(alter.PROV).mean().reset_index()
    tmp.columns = ["PROV", "PROV_05_ALTAF_MEAN"]
    alter_feat = pd.merge(alter_feat, tmp, on="PROV", how="left")
    alter_feat["PROV_05_ALTAF_MEAN_GAP"] = alter_feat["ALTAF_05_mean"] - alter_feat["PROV_05_ALTAF_MEAN"]
    alter_feat = alter_feat.drop(["PROV_05_ALTAF_MEAN"], axis=1)
    
    tmp = alter[alter.ALTERNO == "27"]["ALTBE"].groupby(alter.PROV).mean().reset_index()
    tmp.columns = ["PROV", "PROV_27_ALTBE_MEAN"]
    alter_feat = pd.merge(alter_feat, tmp, on="PROV", how="left")
    alter_feat["PROV_27_ALTBE_MEAN_GAP"] = alter_feat["ALTBE_27_mean"] - alter_feat["PROV_27_ALTBE_MEAN"]
    alter_feat = alter_feat.drop(["PROV_27_ALTBE_MEAN"], axis=1)
    
    tmp = alter[alter.ALTERNO == "05"]["ALTAF"].groupby(alter.PROV).mean().reset_index()
    tmp.columns = ["PROV", "PROV_27_ALTAF_MEAN"]
    alter_feat = pd.merge(alter_feat, tmp, on="PROV", how="left")
    alter_feat["PROV_27_ALTAF_MEAN_GAP"] = alter_feat["ALTAF_27_mean"] - alter_feat["PROV_27_ALTAF_MEAN"]
    alter_feat = alter_feat.drop(["PROV_27_ALTAF_MEAN"], axis=1)
    
    tmp = alter[(alter.ALTERNO == "05") | (alter.ALTERNO == "27")]["ALTBE"].groupby(alter.PROV).mean().reset_index()
    tmp.columns = ["PROV", "PROV_05_27_ALTBE_MEAN"]
    alter_feat = pd.merge(alter_feat, tmp, on="PROV", how="left")
    alter_feat["PROV_05_27_ALTBE_MEAN_GAP"] = alter_feat["ALTBE_05_27_mean"] - alter_feat["PROV_05_27_ALTBE_MEAN"]
    alter_feat = alter_feat.drop(["PROV_05_27_ALTBE_MEAN"], axis=1)
    
    tmp = alter[(alter.ALTERNO == "05") | (alter.ALTERNO == "27")]["ALTAF"].groupby(alter.PROV).mean().reset_index()
    tmp.columns = ["PROV", "PROV_05_27_ALTAF_MEAN"]
    alter_feat = pd.merge(alter_feat, tmp, on="PROV", how="left")
    alter_feat["PROV_05_27_ALTAF_MEAN_GAP"] = alter_feat["ALTAF_05_27_mean"] - alter_feat["PROV_05_27_ALTAF_MEAN"]
    alter_feat = alter_feat.drop(["PROV_05_27_ALTAF_MEAN"], axis=1)
    
    alter_feat = alter_feat.drop(["PROV"], axis=1)
    
    
    return alter_feat

alter_feat = get_alter_feature()
alter_feat.head()

,EID,ALTER_COUNT,ALTER_UNIQUE_COUNT,ALTERNO_01,ALTERNO_02,ALTERNO_03,ALTERNO_04,ALTERNO_05,ALTERNO_10,ALTERNO_12,...,HY_27_ALTBE_MEAN_GAP,HY_27_ALTAF_MEAN_GAP,HY_05_27_ALTBE_MEAN_GAP,HY_05_27_ALTAF_MEAN_GAP,PROV_05_ALTBE_MEAN_GAP,PROV_05_ALTAF_MEAN_GAP,PROV_27_ALTBE_MEAN_GAP,PROV_27_ALTAF_MEAN_GAP,PROV_05_27_ALTBE_MEAN_GAP,PROV_05_27_ALTAF_MEAN_GAP
0,p10000,1,1,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,p100004,2,2,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,-75.580488,-218.487805,-1778.912922,-3209.295038,NaN,NaN,-1497.979295,-2764.089933
2,p100007,1,1,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,p100010,4,4,NaN,NaN,NaN,1.0,1.0,1.0,1.0,...,NaN,NaN,-281.214696,-671.707335,-1785.912922,-3299.295038,NaN,NaN,-1504.979295,-2854.089933
4,p100016,1,1,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,-5490.582511,-7173.754900,-1778.912922,-3191.295038,NaN,NaN,-1497.979295,-2746.089933


In [6]:
def get_branch_feature():
    branch = branch_df.copy()
    
    branch["BRANCH_LIVE_GAP"] = branch["B_ENDYEAR"] - branch["B_REYEAR"] 
    
    branch_feat = branch["TYPECODE"].groupby(branch.EID).count().reset_index()
    branch_feat.columns = ["EID", "BRANCH_COUNT"]
    
    tmp = branch['IFHOME'].groupby(branch.EID).sum().reset_index()
    tmp.columns = ["EID", "BRANCH_HOME_COUNT"]
    branch_feat = pd.merge(branch_feat, tmp, on="EID", how="left")
    
    branch_feat["HOME_RATE"] = branch_feat["BRANCH_HOME_COUNT"] / branch_feat["BRANCH_COUNT"]
    
    tmp = branch["B_REYEAR"].groupby(branch.EID).agg([min, max, np.ptp, np.std]).reset_index()
    tmp.columns = [i if i == 'EID' else 'BRANCH_REYEAR_'+str(i) for i in tmp.columns]
    branch_feat = pd.merge(branch_feat, tmp, on="EID", how="left")
    
    tmp = branch["B_ENDYEAR"].groupby(branch.EID).agg([min, max, np.ptp, np.std]).reset_index()
    tmp.columns = [i if i == 'EID' else 'BRANCH_ENDYEAR_'+str(i) for i in tmp.columns]
    branch_feat = pd.merge(branch_feat, tmp, on="EID", how="left")
    
    tmp = branch["B_ENDYEAR"].groupby(branch.EID).count().reset_index()
    tmp.columns = ["EID", "BRANCH_CLOSE_COUNT"]
    branch_feat = pd.merge(branch_feat, tmp, on="EID", how="left")
    
    branch_feat["CLOSE_RATE"] = branch_feat["BRANCH_CLOSE_COUNT"] / branch_feat["BRANCH_COUNT"]
    
    tmp = branch[branch.B_ENDYEAR.notnull()]["BRANCH_LIVE_GAP"].groupby(branch.EID).agg([min, max, np.mean]).reset_index()
    tmp.columns = [i if i == 'EID' else 'BRANCH_LIVE_GAP_'+str(i) for i in tmp.columns]
    branch_feat = pd.merge(branch_feat, tmp, on="EID", how="left")
    
    return branch_feat

branch_feat = get_branch_feature()
branch_feat.head()

,EID,BRANCH_COUNT,BRANCH_HOME_COUNT,HOME_RATE,BRANCH_REYEAR_min,BRANCH_REYEAR_max,BRANCH_REYEAR_ptp,BRANCH_REYEAR_std,BRANCH_ENDYEAR_min,BRANCH_ENDYEAR_max,BRANCH_ENDYEAR_ptp,BRANCH_ENDYEAR_std,BRANCH_CLOSE_COUNT,CLOSE_RATE,BRANCH_LIVE_GAP_min,BRANCH_LIVE_GAP_max,BRANCH_LIVE_GAP_mean
0,p100035,1,1,1.0,2014,2014,0,NaN,NaN,NaN,NaN,NaN,0,0.000000,NaN,NaN,NaN
1,p100037,1,1,1.0,2013,2013,0,NaN,NaN,NaN,NaN,NaN,0,0.000000,NaN,NaN,NaN
2,p100118,2,2,1.0,2008,2008,0,0.000000,2011.0,2011.0,0.0,NaN,1,0.500000,3.0,3.0,3.0
3,p100164,19,19,1.0,2010,2015,5,1.887168,2015.0,2015.0,0.0,NaN,1,0.052632,4.0,4.0,4.0
4,p100187,7,7,1.0,2013,2014,1,0.534522,NaN,NaN,NaN,NaN,0,0.000000,NaN,NaN,NaN


In [7]:
def get_invest_feature():
    invest = invest_df.copy()
    
    invest["INVEST_LIVE_GAP"] = invest["BTENDYEAR"] - invest["BTYEAR"]
    
    invest_feat = invest["BTEID"].groupby(invest.EID).count().reset_index()
    invest_feat.columns = ["EID", "INVEST_COUNT"]
    
    tmp = invest["IFHOME"].groupby(invest.EID).sum().reset_index()
    tmp.columns = ["EID", "INVEST_HOME_COUNT"]
    invest_feat = pd.merge(invest_feat, tmp, on="EID", how="left")
    
    invest_feat["INVEST_HOME_RATE"] = invest_feat["INVEST_HOME_COUNT"] / invest_feat["INVEST_HOME_COUNT"]
    
    tmp = invest["BTBL"].groupby(invest.EID).agg([min, max, np.mean, np.ptp, np.std]).reset_index()
    tmp.columns = [i if i == 'EID' else 'INVEST_BTBL_'+str(i) for i in tmp.columns]
    invest_feat = pd.merge(invest_feat, tmp, on="EID", how="left")
    
    tmp = invest["BTYEAR"].groupby(invest.EID).agg([min, max, np.ptp, np.std]).reset_index()
    tmp.columns = [i if i == 'EID' else 'INVEST_BTYEAR_'+str(i) for i in tmp.columns]
    invest_feat = pd.merge(invest_feat, tmp, on="EID", how="left")
    
    tmp = invest["BTENDYEAR"].groupby(invest.EID).agg([min, max, np.ptp, np.std]).reset_index()
    tmp.columns = [i if i == 'EID' else 'INVEST_BTENDYEAR_'+str(i) for i in tmp.columns]
    invest_feat = pd.merge(invest_feat, tmp, on="EID", how="left")
    
    tmp = invest["BTENDYEAR"].groupby(invest.EID).count().reset_index()
    tmp.columns = ["EID", "INVEST_CLOSE_COUNT"]
    invest_feat = pd.merge(invest_feat, tmp, on="EID", how="left")
    
    invest_feat["INVEST_CLOSE_RATE"] = invest_feat["INVEST_CLOSE_COUNT"] / invest_feat["INVEST_COUNT"]
    
    tmp = invest[invest.BTENDYEAR.notnull()]["INVEST_LIVE_GAP"].groupby(invest.EID).agg([min, max, np.mean]).reset_index()
    tmp.columns = [i if i == 'EID' else 'INVEST_LIVE_GAP_'+str(i) for i in tmp.columns]
    invest_feat = pd.merge(invest_feat, tmp, on="EID", how="left")
    
    #####################################################################################
    
    tmp = invest["EID"].groupby(invest.BTEID).count().reset_index()
    tmp.columns = ["EID", "INVESTED_COUNT"]
    invest_feat = pd.merge(invest_feat, tmp, on="EID", how="left")
    
    tmp = invest["IFHOME"].groupby(invest.BTEID).sum().reset_index()
    tmp.columns = ["EID", "INVESTED_HOME_COUNT"]
    invest_feat = pd.merge(invest_feat, tmp, on="EID", how="left")
    
    invest_feat["INVESTED_HOME_RATE"] = invest_feat["INVESTED_HOME_COUNT"] / invest_feat["INVESTED_HOME_COUNT"]
    
    tmp = invest["BTBL"].groupby(invest.BTEID).agg([min, max, np.mean, np.ptp, np.std]).reset_index()
    tmp.columns = ["EID" if i == 'BTEID' else 'INVESTED_BTBL_'+str(i) for i in tmp.columns]
    invest_feat = pd.merge(invest_feat, tmp, on="EID", how="left")
    
    return invest_feat

invest_feat = get_invest_feature()
invest_feat.head()

,EID,INVEST_COUNT,INVEST_HOME_COUNT,INVEST_HOME_RATE,INVEST_BTBL_min,INVEST_BTBL_max,INVEST_BTBL_mean,INVEST_BTBL_ptp,INVEST_BTBL_std,INVEST_BTYEAR_min,...,INVEST_LIVE_GAP_max,INVEST_LIVE_GAP_mean,INVESTED_COUNT,INVESTED_HOME_COUNT,INVESTED_HOME_RATE,INVESTED_BTBL_min,INVESTED_BTBL_max,INVESTED_BTBL_mean,INVESTED_BTBL_ptp,INVESTED_BTBL_std
0,p100164,4,4,1.0,1.000,1.000,1.0000,0.000,0.000000,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,p100184,1,1,1.0,0.033,0.033,0.0330,0.000,NaN,2013,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,p10019,5,3,1.0,0.500,1.000,0.8200,0.500,0.248998,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,p100206,2,2,1.0,0.341,1.000,0.6705,0.659,0.465983,2012,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,p100217,1,1,1.0,0.225,0.225,0.2250,0.000,NaN,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
def get_right_feature():
    right = right_df.copy()
    
    right["ASKDATE"] = right["ASKDATE"].apply(translate_year)
    right["FBDATE"] = right[right.FBDATE.notnull()]["FBDATE"].apply(translate_year)
    right["RIGHT_ASK_FB_GAP"] = right["FBDATE"] - right["ASKDATE"]

    right_feat = right["TYPECODE"].groupby(right.EID).count().reset_index()
    right_feat.columns = ["EID", "RIGHT_COUNT"]
    
    tmp = right["RIGHTTYPE"].groupby([right.EID, right.RIGHTTYPE]).count().unstack().reset_index()#.fillna(0)
    tmp.columns = [i if i == 'EID' else 'RIGHTTYPE_'+str(i) for i in tmp.columns]
    right_feat = pd.merge(right_feat, tmp, on='EID', how='left')
    for column in tmp.columns:
        if column != "EID":
            right_feat[column + "_RATE"] = right_feat[column] / right_feat["RIGHT_COUNT"]
    
    tmp = right["ASKDATE"].groupby(right.EID).agg([min, max, np.ptp, np.std]).reset_index()
    tmp.columns = [i if i == 'EID' else 'RIGHT_ASKDATE_'+str(i) for i in tmp.columns]
    right_feat = pd.merge(right_feat, tmp, on="EID", how="left")
    
    tmp = right["FBDATE"].groupby(right.EID).agg([min, max, np.ptp, np.std]).reset_index()
    tmp.columns = [i if i == 'EID' else 'RIGHT_FBDATE_'+str(i) for i in tmp.columns]
    right_feat = pd.merge(right_feat, tmp, on="EID", how="left")
    
    tmp = right[right.FBDATE.notnull()]["RIGHT_ASK_FB_GAP"].groupby(right.EID).agg([min, max, np.mean]).reset_index()
    tmp.columns = [i if i == 'EID' else 'RIGHT_ASK_FB_GAP_'+str(i) for i in tmp.columns]
    right_feat = pd.merge(right_feat, tmp, on="EID", how="left")
    
    return right_feat

right_feat = get_right_feature()
right_feat.head()

,EID,RIGHT_COUNT,RIGHTTYPE_11,RIGHTTYPE_12,RIGHTTYPE_20,RIGHTTYPE_30,RIGHTTYPE_40,RIGHTTYPE_50,RIGHTTYPE_60,RIGHTTYPE_11_RATE,...,RIGHT_ASKDATE_max,RIGHT_ASKDATE_ptp,RIGHT_ASKDATE_std,RIGHT_FBDATE_min,RIGHT_FBDATE_max,RIGHT_FBDATE_ptp,RIGHT_FBDATE_std,RIGHT_ASK_FB_GAP_min,RIGHT_ASK_FB_GAP_max,RIGHT_ASK_FB_GAP_mean
0,p100000,9,NaN,NaN,NaN,NaN,7.0,NaN,2.0,NaN,...,61,35,14.449145,40.0,71.0,31.0,13.038405,0.0,16.0,9.500000
1,p100001,6,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,...,48,0,0.000000,48.0,48.0,0.0,0.000000,0.0,0.0,0.000000
2,p10001,3,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,61,0,0.000000,61.0,61.0,0.0,0.000000,0.0,0.0,0.000000
3,p100014,1,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,35,0,NaN,56.0,56.0,0.0,NaN,21.0,21.0,21.000000
4,p100022,9,NaN,NaN,NaN,NaN,4.0,NaN,5.0,NaN,...,56,5,2.522124,56.0,65.0,9.0,4.743416,0.0,14.0,6.111111


In [9]:
def get_project_feature():
    project = project_df.copy()

    project["DJDATE"] = project["DJDATE"].apply(translate_year)
    
    project_feat = project["TYPECODE"].groupby(project.EID).count().reset_index()
    project_feat.columns = ["EID", "PROJ_COUNT"]
    
    tmp = project["DJDATE"].groupby(project.EID).agg([min, max, np.ptp, np.std]).reset_index()
    tmp.columns = [i if i == 'EID' else 'PROJ_DJDATE_'+str(i) for i in tmp.columns]
    project_feat = pd.merge(project_feat, tmp, on="EID", how="left")
    
    tmp = project["IFHOME"].groupby(project.EID).sum().reset_index()
    tmp.columns = ["EID", "PROJ_HOME_COUNT"]
    project_feat = pd.merge(project_feat, tmp, on="EID", how="left")
    
    project_feat["PROJ_HOME_RATE"] = project_feat["PROJ_COUNT"] / project_feat["PROJ_HOME_COUNT"]
    
    return project_feat

project_feat = get_project_feature()
project_feat.head()

,EID,PROJ_COUNT,PROJ_DJDATE_min,PROJ_DJDATE_max,PROJ_DJDATE_ptp,PROJ_DJDATE_std,PROJ_HOME_COUNT,PROJ_HOME_RATE
0,p100098,1,67,67,0,NaN,1,1.0
1,p100187,1,68,68,0,NaN,1,1.0
2,p100314,2,63,66,3,2.121320,2,1.0
3,p100435,1,59,59,0,NaN,1,1.0
4,p100450,6,50,68,18,5.958188,6,1.0


In [10]:
def translate_date(x):
    x = x.replace('年','')
    x = x.replace('月', '')
    
    year = int(x[:4])
    month = int(x[4:])
    return (2015-year)*12 - month

def get_lawsuit_feature():
    lawsuit = lawsuit_df.copy()
    
    lawsuit["LAWDATE"] = lawsuit["LAWDATE"].apply(translate_date)
    
    lawsuit_feat = lawsuit["TYPECODE"].groupby(lawsuit.EID).count().reset_index()
    lawsuit_feat.columns = ["EID", "LAWSUIT_COUNT"]
    
    tmp = lawsuit["LAWDATE"].groupby(lawsuit.EID).agg([min, max, np.ptp, np.std]).reset_index()
    tmp.columns = [i if i == 'EID' else 'LAWDATE_'+str(i) for i in tmp.columns]
    lawsuit_feat = pd.merge(lawsuit_feat, tmp, on="EID", how="left")
    
    tmp = lawsuit["LAWAMOUNT"].groupby(lawsuit.EID).agg([sum, min, max, np.mean, np.ptp, np.std]).reset_index()
    tmp.columns = [i if i == 'EID' else 'LAWAMOUNT_'+str(i) for i in tmp.columns]
    lawsuit_feat = pd.merge(lawsuit_feat, tmp, on="EID", how="left")
    
    tmp = lawsuit["LAWAMOUNT"].groupby(lawsuit.EID).count().reset_index()
    tmp.columns = ["EID", "LAW_HAS_AMOUNT_COUNT"]
    lawsuit_feat = pd.merge(lawsuit_feat, tmp, on="EID", how="left")
    
    lawsuit_feat["LAW_HAS_AMOUNT_RATE"] = lawsuit_feat["LAW_HAS_AMOUNT_COUNT"] /  lawsuit_feat["LAWSUIT_COUNT"]
    
    return lawsuit_feat

lawsuit_feat = get_lawsuit_feature()
lawsuit_feat.head()

,EID,LAWSUIT_COUNT,LAWDATE_min,LAWDATE_max,LAWDATE_ptp,LAWDATE_std,LAWAMOUNT_sum,LAWAMOUNT_min,LAWAMOUNT_max,LAWAMOUNT_mean,LAWAMOUNT_ptp,LAWAMOUNT_std,LAW_HAS_AMOUNT_COUNT,LAW_HAS_AMOUNT_RATE
0,p100022,2,4,4,0,0.000000,122000,61000,61000,6.100000e+04,0,0.000000e+00,2,1.0
1,p100048,2,0,0,0,0.000000,22000,11000,11000,1.100000e+04,0,0.000000e+00,2,1.0
2,p100074,3,-6,-3,3,1.527525,8646200,17100,5509600,2.882067e+06,5492500,2.753937e+06,3,1.0
3,p100084,2,11,11,0,0.000000,1488400,744200,744200,7.442000e+05,0,0.000000e+00,2,1.0
4,p100107,2,16,16,0,0.000000,2,1,1,1.000000e+00,0,0.000000e+00,2,1.0


In [11]:
def get_breakfaith_feature():
    breakfaith = breakfaith_df.copy()
    
    breakfaith["FBDATE"] = breakfaith["FBDATE"].apply(translate_date)
    breakfaith["SXENDDATE"] = breakfaith[breakfaith.SXENDDATE.notnull()]["SXENDDATE"].apply(translate_date)
    breakfaith["BREAKFAITH_SXDATE_GAP"] = breakfaith["SXENDDATE"] - breakfaith["FBDATE"]
    
    
    breakfaith_feat = breakfaith["TYPECODE"].groupby(breakfaith.EID).count().reset_index()
    breakfaith_feat.columns = ["EID", "BREAKFAITH_COUNT"]
    
    tmp = breakfaith["FBDATE"].groupby(breakfaith.EID).agg([min, max, np.ptp, np.std]).reset_index()
    tmp.columns = [i if i == 'EID' else 'FBDATE_'+str(i) for i in tmp.columns]
    breakfaith_feat = pd.merge(breakfaith_feat, tmp, on="EID", how="left")

    tmp = breakfaith["SXENDDATE"].groupby(breakfaith.EID).agg([min, max, np.ptp, np.std]).reset_index()
    tmp.columns = [i if i == 'EID' else 'SXENDDATE_'+str(i) for i in tmp.columns]
    breakfaith_feat = pd.merge(breakfaith_feat, tmp, on="EID", how="left")

    tmp = breakfaith["SXENDDATE"].groupby(breakfaith.EID).count().reset_index()
    tmp.columns = ["EID", "BREAKFAITH_SXEND_COUNT"]
    breakfaith_feat = pd.merge(breakfaith_feat, tmp, on="EID", how="left")
    
    breakfaith_feat["BREAKFAITH_SXEND_RATE"] = breakfaith_feat["BREAKFAITH_SXEND_COUNT"] / breakfaith_feat["BREAKFAITH_COUNT"]
    
    tmp = breakfaith[breakfaith.SXENDDATE.notnull()]["BREAKFAITH_SXDATE_GAP"].groupby(breakfaith.EID).agg([min, max, np.mean]).reset_index()
    tmp.columns = [i if i == 'EID' else 'BREAKFAITH_SXDATE_GAP_'+str(i) for i in tmp.columns]
    breakfaith_feat = pd.merge(breakfaith_feat, tmp, on="EID", how="left")
    
    return breakfaith_feat

breakfaith_feat = get_breakfaith_feature()
breakfaith_feat.head()

,EID,BREAKFAITH_COUNT,FBDATE_min,FBDATE_max,FBDATE_ptp,FBDATE_std,SXENDDATE_min,SXENDDATE_max,SXENDDATE_ptp,SXENDDATE_std,BREAKFAITH_SXEND_COUNT,BREAKFAITH_SXEND_RATE,BREAKFAITH_SXDATE_GAP_min,BREAKFAITH_SXDATE_GAP_max,BREAKFAITH_SXDATE_GAP_mean
0,p100048,1,-5,-5,0,NaN,7.0,7.0,0.0,NaN,1,1.0,12.0,12.0,12.0
1,p100344,8,-7,-5,2,0.886405,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN
2,p100347,10,-6,-2,4,2.065591,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN
3,p100667,3,-7,-7,0,0.000000,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN
4,p101171,2,-5,-5,0,0.000000,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN


In [12]:
def transform_pnum(x):
    if x == 0:
        return 0
    if x == '若干':
        return 1
    elif x[-3:] == '人':
        return float(x[:-3])
    else:
        return float(x)

def get_recruit_feature():
    recruit = recruit_df.copy()
    
    recruit.PNUM.fillna(0, inplace=True)
    recruit['RECR_YEAR_GAP'] = 2017 - pd.to_datetime(recruit.RECDATE).dt.year
    recruit["PNUM"] = recruit["PNUM"].apply(transform_pnum)
    recruit["RECDATE"] = recruit["RECDATE"].apply(translate_year)
    
    recruit_feat = recruit["PNUM"].groupby(recruit.EID).agg([len, sum]).reset_index()
    recruit_feat.columns = ["EID", "RECRUIT_TIMES", "RECRUIT_COUNT"]
    
    tmp = recruit["RECDATE"].groupby(recruit.EID).agg([min, max, np.ptp, np.std]).reset_index()
    tmp.columns = [i if i == 'EID' else 'RECDATE_'+str(i) for i in tmp.columns]
    recruit_feat = pd.merge(recruit_feat, tmp, on="EID", how="left")
    
    tmp = recruit[(recruit.RECR_YEAR_GAP != 9) & (recruit.RECR_YEAR_GAP != 5)].groupby(["EID", "RECR_YEAR_GAP"])["PNUM"].sum().unstack().reset_index()
    tmp["4_BI_3"] = (tmp[3] - tmp[4]) / tmp[4]
    tmp["3_BI_2"] = (tmp[2] - tmp[3]) / tmp[3]
    tmp.drop([2, 3, 4], axis=1, inplace=True)
    tmp.columns = (i if i == 'EID' else 'RECRUIT_INCREATE_RATE_'+ str(i) for i in tmp.columns)
    recruit_feat = pd.merge(recruit_feat, tmp, on="EID", how="left")
    
    tmp = recruit.groupby(['EID', 'WZCODE'])['PNUM'].sum().unstack().reset_index()
    tmp.columns = [i if i == 'EID' else 'WZCODE_'+str(i) for i in tmp.columns]
    recruit_feat = pd.merge(recruit_feat, tmp, on='EID', how='left')
    
    return recruit_feat

recruit_feat = get_recruit_feature()
recruit_feat.head()

,EID,RECRUIT_TIMES,RECRUIT_COUNT,RECDATE_min,RECDATE_max,RECDATE_ptp,RECDATE_std,RECRUIT_INCREATE_RATE_4_BI_3,RECRUIT_INCREATE_RATE_3_BI_2,WZCODE_zp01,WZCODE_zp02,WZCODE_zp03
0,p100004,9.0,32.0,60,68,8,4.216370,NaN,0.461538,NaN,8.0,24.0
1,p100008,9.0,9.0,58,68,10,5.019407,NaN,0.250000,4.0,NaN,5.0
2,p100032,4.0,6.0,55,55,0,0.000000,NaN,NaN,NaN,NaN,6.0
3,p100037,11.0,27.0,53,64,11,4.358899,NaN,-0.650000,1.0,4.0,22.0
4,p100038,4.0,4.0,59,67,8,4.618802,NaN,0.000000,NaN,1.0,3.0


In [13]:
def to_month(x):
    x = x.decode("gbk")
    x = x.replace(u'年','')
    x = x.replace(u'月', '')
    
    year = int(x[:4])
    month = int(x[4:])
    return (2015-year)*12 - month
    
def get_qualification_feature():
    qual = qualification_df.copy()
        
    qual["BEGINDATE"] = qual["BEGINDATE"].apply(to_month)
    qual["EXPIRYDATE"] = qual[qual.EXPIRYDATE.notnull()]["EXPIRYDATE"].apply(to_month)
    qual["QUAL_DATE_GAP"] = qual["EXPIRYDATE"] - qual["BEGINDATE"]
    
    qual_feat = qual["ADDTYPE"].groupby(qual.EID).count().reset_index()
    qual_feat.columns = ["EID", "QUAL_COUNT"]
    
    tmp = qual["ADDTYPE"].groupby([qual.EID, qual.ADDTYPE]).count().unstack().reset_index()#.fillna(0)
    tmp.columns = [i if i == 'EID' else 'QUAL_ADDTYPE_'+str(i) for i in tmp.columns]
    qual_feat = pd.merge(qual_feat, tmp, on='EID', how='left')
    
    tmp = qual["BEGINDATE"].groupby(qual.EID).agg([min, max, np.ptp, np.std]).reset_index()
    tmp.columns = [i if i == 'EID' else 'QUAL_BEGINDATE_'+str(i) for i in tmp.columns]
    qual_feat = pd.merge(qual_feat, tmp, on="EID", how="left")
    
    tmp = qual["EXPIRYDATE"].groupby(qual.EID).agg([min, max, np.ptp, np.std]).reset_index()
    tmp.columns = [i if i == 'EID' else 'QUAL_EXPIRYDATE_'+str(i) for i in tmp.columns]
    qual_feat = pd.merge(qual_feat, tmp, on="EID", how="left")
    
    tmp = qual[qual["EXPIRYDATE"].notnull()]["QUAL_DATE_GAP"].groupby(qual.EID).agg([min, max, np.mean]).reset_index()
    tmp.columns = [i if i == 'EID' else 'QUAL_DATE_GAP_'+str(i) for i in tmp.columns]
    qual_feat = pd.merge(qual_feat, tmp, on="EID", how="left")
    
    tmp = qual["EXPIRYDATE"].groupby(qual.EID).count().reset_index()
    tmp.columns = ["EID", "QUAL_HAS_EXPIRYDATE_COUNT"]
    qual_feat = pd.merge(qual_feat, tmp, on="EID", how="left")
    
    qual_feat["QUAL_HAS_EXPIRYDATE_RATE"] = qual_feat["QUAL_HAS_EXPIRYDATE_COUNT"] / qual_feat["QUAL_COUNT"]
    
    return qual_feat

qual_feat = get_qualification_feature()
qual_feat.head()

,EID,QUAL_COUNT,QUAL_ADDTYPE_1,QUAL_ADDTYPE_2,QUAL_ADDTYPE_3,QUAL_BEGINDATE_min,QUAL_BEGINDATE_max,QUAL_BEGINDATE_ptp,QUAL_BEGINDATE_std,QUAL_EXPIRYDATE_min,QUAL_EXPIRYDATE_max,QUAL_EXPIRYDATE_ptp,QUAL_EXPIRYDATE_std,QUAL_DATE_GAP_min,QUAL_DATE_GAP_max,QUAL_DATE_GAP_mean,QUAL_HAS_EXPIRYDATE_COUNT,QUAL_HAS_EXPIRYDATE_RATE
0,p100533,1,1.0,NaN,NaN,-8,-8,0,NaN,-68.0,-68.0,0.0,NaN,-60.0,-60.0,-60.0,1,1.0
1,p100847,1,1.0,NaN,NaN,67,67,0,NaN,7.0,7.0,0.0,NaN,-60.0,-60.0,-60.0,1,1.0
2,p10086,1,1.0,NaN,NaN,19,19,0,NaN,-41.0,-41.0,0.0,NaN,-60.0,-60.0,-60.0,1,1.0
3,p100916,4,NaN,1.0,3.0,5,53,48,20.870633,-30.0,17.0,47.0,23.5,-36.0,-11.0,-27.0,4,1.0
4,p101264,1,NaN,1.0,NaN,2,2,0,NaN,-34.0,-34.0,0.0,NaN,-36.0,-36.0,-36.0,1,1.0


In [14]:
train = pd.read_csv("./data/train.csv")
labels = train.TARGET.values

test = pd.read_csv("./data/evaluation_public.csv")
train.head()

,EID,TARGET,ENDDATE
0,p1,0,NaN
1,p100000,0,NaN
2,p100002,0,NaN
3,p100004,0,NaN
4,p100005,0,NaN


In [15]:
train_feat = pd.merge(train[["EID"]], entbase_feat, on="EID", how="left")
train_feat = pd.merge(train_feat, alter_feat, on="EID", how="left")
train_feat = pd.merge(train_feat, branch_feat, on="EID", how="left")
train_feat = pd.merge(train_feat, invest_feat, on="EID", how="left")
train_feat = pd.merge(train_feat, right_feat, on="EID", how="left")
train_feat = pd.merge(train_feat, project_feat, on="EID", how="left")
train_feat = pd.merge(train_feat, lawsuit_feat, on="EID", how="left")
train_feat = pd.merge(train_feat, breakfaith_feat, on="EID", how="left")
train_feat = pd.merge(train_feat, recruit_feat, on="EID", how="left")
train_feat = pd.merge(train_feat, qual_feat, on="EID", how="left")

train_feat.head()

,EID,PROV,RGYEAR,HY,ZCZB,ETYPE,MPNUM,INUM,ENUM,FINZB,...,QUAL_BEGINDATE_std,QUAL_EXPIRYDATE_min,QUAL_EXPIRYDATE_max,QUAL_EXPIRYDATE_ptp,QUAL_EXPIRYDATE_std,QUAL_DATE_GAP_min,QUAL_DATE_GAP_max,QUAL_DATE_GAP_mean,QUAL_HAS_EXPIRYDATE_COUNT,QUAL_HAS_EXPIRYDATE_RATE
0,p1,11,2,51.0,10000.0,6,6.0,3.0,3.0,15000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,p100000,11,3,51.0,10.0,7,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,p100002,11,5,19.0,10.0,7,0.0,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,p100004,11,2,82.0,100.0,7,1.0,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,p100005,11,6,72.0,3.0,7,1.0,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
test_feat = pd.merge(test, entbase_feat, on="EID", how="left")
test_feat = pd.merge(test_feat, alter_feat, on="EID", how="left")
test_feat = pd.merge(test_feat, branch_feat, on="EID", how="left")
test_feat = pd.merge(test_feat, invest_feat, on="EID", how="left")
test_feat = pd.merge(test_feat, right_feat, on="EID", how="left")
test_feat = pd.merge(test_feat, project_feat, on="EID", how="left")
test_feat = pd.merge(test_feat, lawsuit_feat, on="EID", how="left")
test_feat = pd.merge(test_feat, breakfaith_feat, on="EID", how="left")
test_feat = pd.merge(test_feat, recruit_feat, on="EID", how="left")
test_feat = pd.merge(test_feat, qual_feat, on="EID", how="left")

test_feat.head()

,EID,PROV,RGYEAR,HY,ZCZB,ETYPE,MPNUM,INUM,ENUM,FINZB,...,QUAL_BEGINDATE_std,QUAL_EXPIRYDATE_min,QUAL_EXPIRYDATE_max,QUAL_EXPIRYDATE_ptp,QUAL_EXPIRYDATE_std,QUAL_DATE_GAP_min,QUAL_DATE_GAP_max,QUAL_DATE_GAP_mean,QUAL_HAS_EXPIRYDATE_COUNT,QUAL_HAS_EXPIRYDATE_RATE
0,p10000,11,4,51.0,10.0,6,1.0,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,p100001,11,4,39.0,50.0,7,0.0,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,p100006,11,0,18.0,100.0,7,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,p100009,11,0,52.0,5.0,8,1.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,p10001,11,1,51.0,100.0,7,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
train_count = train_feat.shape[0]
test_count = test_feat.shape[0]

all_feat = pd.concat([train_feat, test_feat]).reset_index(drop=True)

all_feat["ALTER_COUNT_MEAN_YEAR"] = all_feat["ALTER_COUNT"] / all_feat["RGYEAR"]
all_feat["BRANCH_COUNT_MEAN_YEAR"] = all_feat["BRANCH_COUNT"] / all_feat["RGYEAR"]
all_feat["RIGHT_COUNT_MEAN_YEAR"] = all_feat["RIGHT_COUNT"] / all_feat["RGYEAR"]
all_feat["PROJ_COUNT_MEAN_YEAR"] = all_feat["PROJ_COUNT"] / all_feat["RGYEAR"]
all_feat["LAWSUIT_COUNT_MEAN_YEAR"] = all_feat["LAWSUIT_COUNT"] / all_feat["RGYEAR"]
all_feat["SX_COUNT_MEAN_YEAR"] = all_feat["BREAKFAITH_COUNT"] / all_feat["RGYEAR"]
all_feat["RECRUIT_COUNT_MEAN_YEAR"] = all_feat["RECRUIT_COUNT"] / all_feat["RGYEAR"]
all_feat["RECRUIT_TIMES_MEAN_YEAR"] = all_feat["RECRUIT_TIMES"] / all_feat["RGYEAR"]
all_feat["INDEX_SUM_MEAN_YEAR"] = all_feat["INDEX_SUM"] / all_feat["RGYEAR"]

# all_feat["ALTER_COUNT_MEAN_YEAR1"] = all_feat["ALTER_COUNT"] / all_feat["RGYEAR"]
# all_feat["BRANCH_COUNT_MEAN_YEAR1"] = all_feat["BRANCH_COUNT"] / all_feat["RGYEAR"]
# all_feat["RIGHT_COUNT_MEAN_YEAR1"] = all_feat["RIGHT_COUNT"] / all_feat["RGYEAR"]
# all_feat["PROJ_COUNT_MEAN_YEAR1"] = all_feat["PROJ_COUNT"] / all_feat["RGYEAR"]
# all_feat["LAWSUIT_COUNT_MEAN_YEAR1"] = all_feat["LAWSUIT_COUNT"] / all_feat["RGYEAR"]
# all_feat["SX_COUNT_MEAN_YEAR1"] = all_feat["BREAKFAITH_COUNT"] / all_feat["RGYEAR"]
# all_feat["RECRUIT_COUNT_MEAN_YEAR1"] = all_feat["RECRUIT_COUNT"] / all_feat["RGYEAR"]
# all_feat["RECRUIT_TIMES_MEAN_YEAR1"] = all_feat["RECRUIT_TIMES"] / all_feat["RGYEAR"]
# all_feat["INDEX_SUM_MEAN_YEAR1"] = all_feat["INDEX_SUM"] / all_feat["RGYEAR"]

all_feat.RIGHT_COUNT.fillna(0, inplace=True)
all_feat["RGYEAR_RIGHT_NUM"] = get_interaction_feature(all_feat, "RGYEAR", "RIGHT_COUNT")

all_feat.ALTER_COUNT.fillna(0, inplace=True)
all_feat["RGYEAR_ALTER_COUNT"] = get_interaction_feature(all_feat, "RGYEAR", "ALTER_COUNT")

all_feat.RECRUIT_COUNT.fillna(0, inplace=True)
all_feat["RGYEAR_RECRUIT_COUNT"] = get_interaction_feature(all_feat, "RGYEAR", "RECRUIT_COUNT")

# for column in ["ALTER_COUNT", "RIGHT_COUNT"]:

for column in ["ALTER_COUNT", "RIGHT_COUNT"]:
    #groupby_list = [["HY"], ["ETYPE"], ["HY", "ETYPE"], ["HY", "PROV"], ["ETYPE", "PROV"]]
    groupby_list = [["HY"], ["HY", "PROV"],["PROV"]]
    for groupby in groupby_list:
        groupby_keylist = []
        for key in groupby:
            groupby_keylist.append(all_feat[key])

        tmp = all_feat[column].groupby(groupby_keylist).mean().reset_index()
        tmp.columns = np.append(groupby, ["MEAN_COLUMN"])
        all_feat = pd.merge(all_feat, tmp, on=groupby, how="left")
        all_feat["_".join(groupby) + "_" + column + "_MEAN_GAP"] = all_feat[column] - all_feat["MEAN_COLUMN"]
    #             if column in ["ZCZB", "FINZB"]:
    #                 all_feat["_".join(groupby) + "_" + column + "_RATE"] = all_feat[column] / all_feat["MEAN_COLUMN"]
        all_feat.drop(["MEAN_COLUMN"], axis=1, inplace=True)

        tmp = all_feat[column].groupby(groupby_keylist).min().reset_index()
        tmp.columns = np.append(groupby, ["MIN_COLUMN"])
        all_feat = pd.merge(all_feat, tmp, on=groupby, how="left")
        all_feat["_".join(groupby) + "_" + column + "_MIN_GAP"] = all_feat[column] - all_feat["MIN_COLUMN"]
        all_feat.drop(["MIN_COLUMN"], axis=1, inplace=True)

        tmp = all_feat[column].groupby(groupby_keylist).max().reset_index()
        tmp.columns = np.append(groupby, ["MAX_COLUMN"])
        all_feat = pd.merge(all_feat, tmp, on=groupby, how="left")
        all_feat["_".join(groupby) + "_" + column + "_MAX_GAP"] = all_feat[column] - all_feat["MAX_COLUMN"]
        all_feat.drop(["MAX_COLUMN"], axis=1, inplace=True)

        tmp = all_feat[column].groupby(groupby_keylist).sum().reset_index()
        tmp.columns = np.append(groupby, ["SUM_COLUMN"])
        all_feat = pd.merge(all_feat, tmp, on=groupby, how="left")
        all_feat["_".join(groupby) + "_" + column + "_SUM_GAP"] = all_feat[column] / all_feat["SUM_COLUMN"]
        all_feat.drop(["SUM_COLUMN"], axis=1, inplace=True)
        
#all_feat.drop(["HY", "ETYPE", "PROV"], axis=1, inplace=True)

In [18]:
train_feat = all_feat[:train_count].reset_index(drop=True)
test_feat = all_feat[-test_count:].reset_index(drop=True)

# enc = ce.LeaveOneOutEncoder(cols=["HY", "ETYPE"])
# enc.fit(train_feat, labels)
# train_feat = enc.transform(train_feat)
# test_feat = enc.transform(test_feat)

train_feat.head()

,EID,PROV,RGYEAR,HY,ZCZB,ETYPE,MPNUM,INUM,ENUM,FINZB,...,HY_RIGHT_COUNT_MAX_GAP,HY_RIGHT_COUNT_SUM_GAP,HY_PROV_RIGHT_COUNT_MEAN_GAP,HY_PROV_RIGHT_COUNT_MIN_GAP,HY_PROV_RIGHT_COUNT_MAX_GAP,HY_PROV_RIGHT_COUNT_SUM_GAP,PROV_RIGHT_COUNT_MEAN_GAP,PROV_RIGHT_COUNT_MIN_GAP,PROV_RIGHT_COUNT_MAX_GAP,PROV_RIGHT_COUNT_SUM_GAP
0,p1,11,2,51.0,10000.0,6,6.0,3.0,3.0,15000.0,...,-3035.0,0.000000,-1.108613,0.0,-3035.0,0.000000,-1.602031,0.0,-19818.0,0.000000
1,p100000,11,3,51.0,10.0,7,0.0,1.0,0.0,0.0,...,-3026.0,0.000089,7.891387,9.0,-3026.0,0.000136,7.397969,9.0,-19809.0,0.000029
2,p100002,11,5,19.0,10.0,7,0.0,2.0,0.0,0.0,...,-139.0,0.000000,-1.403743,0.0,-139.0,0.000000,-1.602031,0.0,-19818.0,0.000000
3,p100004,11,2,82.0,100.0,7,1.0,2.0,0.0,0.0,...,-65.0,0.000000,-0.510989,0.0,-27.0,0.000000,-1.602031,0.0,-19818.0,0.000000
4,p100005,11,6,72.0,3.0,7,1.0,2.0,0.0,0.0,...,-1411.0,0.000000,-0.940223,0.0,-1010.0,0.000000,-1.602031,0.0,-19818.0,0.000000


In [19]:
#train_feat["TARGET"] = labels
#train_feat.to_csv("./trainFeature_jj.csv", index=False)
#test_feat.to_csv("./testFeature_jj.csv", index=False)
#train_feat.drop(["TARGET"], axis=1, inplace=True)

test_feat['EID'] = test_feat["EID"].apply(lambda x: int(x[1:]))
train_feat['EID'] = train_feat["EID"].apply(lambda x: int(x[1:]))


In [51]:
import time

params={
    'booster':'gbtree',
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    
    'max_depth': 10,
    'subsample': 0.7, #0.7
    'colsample_bytree': 0.7, #0.7,
    'stratified':True,
    'min_child_weight': 15,
    'eta': 0.01,
    'seed': 20,
    'silent':1
}

dtrain = xgb.DMatrix(train_feat, label=labels)
dtest = xgb.DMatrix(test_feat, label=np.zeros(test_feat.shape[0]))

print "start cv:", time.strftime("%H:%M:%S",time.localtime())
res = xgb.cv(params, dtrain, 2000, nfold=5, early_stopping_rounds=50, verbose_eval=10)
print "done cv:", time.strftime("%H:%M:%S",time.localtime())
print "best cv:", res['test-auc-mean'].tail(1).values[0]

start cv: 21:57:56
[0]	train-auc:0.673448+0.0018812	test-auc:0.644652+0.00431034
[10]	train-auc:0.706455+0.000752189	test-auc:0.667716+0.00389641
[20]	train-auc:0.711024+0.000839489	test-auc:0.670333+0.00385041
[30]	train-auc:0.713686+0.00100035	test-auc:0.67139+0.00409615
[40]	train-auc:0.71595+0.00112493	test-auc:0.672365+0.00414644
[50]	train-auc:0.717977+0.00122812	test-auc:0.672979+0.0040914
[60]	train-auc:0.719864+0.0010349	test-auc:0.673706+0.0040056
[70]	train-auc:0.721587+0.00120689	test-auc:0.67432+0.0039601
[80]	train-auc:0.723129+0.00111607	test-auc:0.67488+0.00383418
[90]	train-auc:0.724679+0.00108631	test-auc:0.675306+0.0037105
[100]	train-auc:0.726261+0.0010015	test-auc:0.675901+0.00383524
[110]	train-auc:0.727735+0.000908627	test-auc:0.676379+0.00383625
[120]	train-auc:0.729224+0.000713993	test-auc:0.676933+0.00376646
[130]	train-auc:0.73069+0.000599763	test-auc:0.677363+0.00377566
[140]	train-auc:0.732185+0.000640654	test-auc:0.677869+0.00374368
[150]	train-auc:0.73363

In [ ]:
watchlist = [(dtrain, 'train')]
model = xgb.train(params, dtrain, len(res), watchlist, verbose_eval=10)

In [ ]:
def plot_feature_importance(model):
    df = pd.DataFrame(model.get_fscore().items(), columns=['feature','importance']).sort_values('importance', ascending=False)
    print "feature， importance"
    sum = df['importance'].sum()
    for index, row in df.iterrows():
        print row["feature"], row["importance"]
plot_feature_importance(model)

In [ ]:
ptest = model.predict(dtest)
sub = pd.DataFrame({'EID':test.EID, 'FORTARGET':[1 if i > 0.18 else 0 for i in ptest], 'PROB':ptest})
sub.to_csv('./1209-0.6892068' + ".csv", index=0)